In [1]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px



In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()



In [3]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                 {'label': 'ALL SITES', 'value': 'ALL'},
                                                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                            ],
                                            value='ALL',
                                            placeholder="Select a Launch Site",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 
                                                       2500:'2500',
                                                       5000:'5000',
                                                       7500:'7500', 
                                                       10000: '10000'},
                                                value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])



In [4]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie(site_dropdown):
    if site_dropdown == 'ALL':
        piechart = px.pie(data_frame = spacex_df, names='Launch Site', values='class', title='Total Launches for All Sites')
    else:
        specific_df = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        piechart = px.pie(data_frame = specific_df, names='class', title='Total Launch for a Specific Site')
    return piechart


In [5]:
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id='payload-slider',component_property='value'))

def get_scatter(launch,slider):
    low, high = slider
    if launch == 'ALL':
        space = (spacex_df['Payload Mass (kg)']>low) & (spacex_df['Payload Mass (kg)']<high)
        df = spacex_df[space]
    else:
        data = spacex_df[spacex_df['Launch Site']==launch]
        space = (data['Payload Mass (kg)']>low) & (data['Payload Mass (kg)']<high)
        df = data[space]    
    scatter = px.scatter(df, y='class', x='Payload Mass (kg)', color='Booster Version Category')
    return scatter

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server(host="localhost")

Dash is running on http://localhost:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8050 (Press CTRL+C to quit)
127.0.0.1 - - [11/Jul/2022 17:52:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 17:52:13] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Jul/2022 17:52:13] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Jul/2022 17:52:13] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Jul/2022 17:52:13] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Jul/2022 17:52:13] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 17:52:13] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 17:52:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 17:52:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 17:54:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 17:54:16] "POST /_dash-u